# 地图可视化

## 数据准备

In [1]:
import json
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
import webbrowser

import sys
sys.path.append('..')  # 添加上级目录到路径
from tools.data_tool import *

# 北京市中心坐标
beijing_center = [39.9042, 116.4074]

In [2]:
# 读取数据
with open("../data/house_data.json", 'r') as f:
    data = json.load(f)


houses = json_data_to_vector(data)


# 读取数据
with open("../data/poi_data.json", 'r') as f:
    data = json.load(f)

pois = json_data_to_vector(data)

del data

有效数据点数量: 9487
有效数据点数量: 40887


## 散点图

In [9]:
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

# 创建地图
m = folium.Map(
    location=beijing_center,
    zoom_start=11,
    tiles='OpenStreetMap'
)

# 创建主聚类
marker_cluster = MarkerCluster().add_to(m)

# 创建特征组
house_group = folium.FeatureGroup(name='🏠 房屋', show=True).add_to(m)
poi_group = folium.FeatureGroup(name='📍 POI', show=True).add_to(m)

# 添加房屋标记 - 橙红色正方形
for idx, row in houses.iterrows():
    folium.RegularPolygonMarker(
        location=[row['lat'], row['lon']],
        popup=f"<b>🏠 房屋</b><br>小区: {row.get('community_name', idx)}<br>价格: {row.get('price_per_meter', 'N/A')}元/㎡",
        tooltip=f"房屋 #{idx}",
        number_of_sides=4,  # 正方形
        radius=8,
        color='#FF4500',  # 橙红色边框
        fill=True,
        fillColor='#FF6347',  # 番茄红填充
        fillOpacity=0.8,
        weight=2,
        rotation=45  # 旋转45度变成菱形
    ).add_to(house_group)

# 添加POI标记 - 青色三角形
for idx, row in pois.iterrows():
    poi_name = row.get('tags', {}).get('name', f'POI {idx}') if isinstance(row.get('tags'), dict) else f'POI {idx}'
    
    folium.RegularPolygonMarker(
        location=[row['lat'], row['lon']],
        popup=f"<b>📍 POI</b><br>名称: {poi_name}",
        tooltip=poi_name,
        number_of_sides=3,  # 三角形
        radius=8,
        color='#008B8B',  # 深青色边框
        fill=True,
        fillColor='#20B2AA',  # 浅海绿色填充
        fillOpacity=0.8,
        weight=2,
        rotation=0
    ).add_to(poi_group)

# 添加图层控制
folium.LayerControl(collapsed=False).add_to(m)

# 添加增强型图例
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 200px; height: 150px; 
            background-color: rgba(255, 255, 255, 0.95); 
            border: 3px solid #2C3E50; z-index:9999; 
            font-size: 12px; padding: 12px; border-radius: 10px;
            box-shadow: 0 6px 12px rgba(0,0,0,0.4);">
<p style="margin: 0 0 12px 0; font-weight: bold; font-size: 14px; 
          color: #2C3E50; text-align: center;
          border-bottom: 3px solid #3498DB; padding-bottom: 8px;">
    🗺️ 图例说明
</p>
<div style="display: flex; align-items: center; margin: 8px 0;">
    <div style="width: 20px; height: 20px; background-color: #FF6347; 
                border: 2px solid #FF4500; transform: rotate(45deg);
                margin-right: 12px;"></div>
    <span style="font-weight: 600; font-size: 11px;">房屋位置 (<span style="color: #E74C3C;">{}</span> 个)</span>
</div>
<div style="display: flex; align-items: center; margin: 8px 0;">
    <div style="width: 0; height: 0; 
                border-left: 10px solid transparent;
                border-right: 10px solid transparent;
                border-bottom: 20px solid #20B2AA;
                margin-right: 12px; margin-left: 3px;"></div>
    <span style="font-weight: 600; font-size: 11px;">POI位置 (<span style="color: #16A085;">{}</span> 个)</span>
</div>
<p style="margin-top: 12px; font-size: 10px; color: #7F8C8D; 
          text-align: center; font-style: italic;">
    💡 点击标记查看详细信息
</p>
</div>
'''.format(len(houses), len(pois))

m.get_root().html.add_child(folium.Element(legend_html))

output_file = '../image/scatter_graph_beijing_houses_pois_map.html'

m.save(output_file)
print(f"✓ 几何形状区分地图已保存到: {output_file}")
print(f"  - 🔶 房屋: {len(houses)} 个 (橙红色菱形)")
print(f"  - 🔺 POI: {len(pois)} 个 (青色三角形)")

✓ 几何形状区分地图已保存到: ../image/scatter_graph_beijing_houses_pois_map.html
  - 🔶 房屋: 9487 个 (橙红色菱形)
  - 🔺 POI: 40887 个 (青色三角形)


## 热力图

In [8]:
# 创建地图
m = folium.Map(
    location=beijing_center,
    zoom_start=11,
    tiles='OpenStreetMap'
)

heat_data = []
for idx, row in houses.iterrows():
    heat_data.append([row['lat'], row['lon']])
for idx, row in pois.iterrows():
    heat_data.append([row['lat'], row['lon']])
    
HeatMap(heat_data, radius=15, blur=25, max_zoom=13).add_to(m)

# 保存地图
output_file = '../image/heat_map_beijing_houses_pois_map.html'
m.save(output_file)

webbrowser.open(output_file)

True